In [1]:
import numpy as np 
import pandas as pd
import scipy.constants as const
import os
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

\begin{equation}
\rm{MTTF(T_{joule}) = A_{MTTF} \cdot exp(\frac{E_A}{k_B T_{joule}})}
\end{equation}

\begin{equation}
\rm{P_{failure, rate}(t, T_{joule})=1-exp(-[\frac{t}{MTTF(T_{joule})}]^{\beta})}
\end{equation}

\begin{equation}
\rm{\tau_{lifetime} = MTTF(T_{joule})[-1 \cdot log_{e}(1 - P_{failure, rate}(T_{joule}))]^{1/\beta}}
\end{equation}

In [2]:
num_samples = 1001
heater_temperature = np.linspace(200, 300, num_samples)

dpt_fit = pd.DataFrame([], columns=['heater_temp', 'heater_lifetime', 'probability'])
dpm_fit = pd.DataFrame([], columns=['heater_temp', 'heater_lifetime', 'probability'])
dpb_fit = pd.DataFrame([], columns=['heater_temp', 'heater_lifetime', 'probability'])

dpt = pd.read_csv(os.getcwd()+'/figure_9_dpt.csv')
dpt.columns = ['heater_temp', 'heater_lifetime']
dpt_polyfit = np.polyfit(dpt['heater_temp'], np.log10(dpt['heater_lifetime']), 1)
dpt_polyval = 10**np.polyval(dpt_polyfit, heater_temperature)

dpt_fit['heater_temp'] = heater_temperature
dpt_fit['heater_lifetime'] = dpt_polyval
dpt_fit['probability']  = '1DPT'

dpm = pd.read_csv(os.getcwd()+'/figure_9_dpm.csv')
dpm.columns = ['heater_temp', 'heater_lifetime']
dpm_polyfit = np.polyfit(dpm['heater_temp'], np.log10(dpm['heater_lifetime']), 1)
dpm_polyval = 10**np.polyval(dpm_polyfit, heater_temperature)

dpm_fit['heater_temp'] = heater_temperature
dpm_fit['heater_lifetime'] = dpm_polyval
dpm_fit['probability']  = '1DPM'

dpb = pd.read_csv(os.getcwd()+'/figure_9_dpb.csv')
dpb.columns = ['heater_temp', 'heater_lifetime']
dpb_polyfit = np.polyfit(dpb['heater_temp'], np.log10(dpb['heater_lifetime']), 1)
dpb_polyval = 10**np.polyval(dpb_polyfit, heater_temperature)

dpb_fit['heater_temp'] = heater_temperature
dpb_fit['heater_lifetime'] = dpb_polyval
dpb_fit['probability']  = '1DPB'

P_failure_rate = np.array([1e-3, 1e-6, 1e-9])
A_lifetime = (-1*np.log(1-P_failure_rate))**(1/4)
A_lifetime = (-1*np.log(1-P_failure_rate))**(1/15)
print(A_lifetime)
E_A = 1.94 * const.eV

MTTF_1DPT = dpt_polyval / A_lifetime[0] # / MTTF_exp
MTTF_1DPM = dpm_polyval / A_lifetime[1] #/ MTTF_exp
MTTF_1DPB = dpb_polyval / A_lifetime[2] # / MTTF_exp 

dpt_fit['MTTF'] = MTTF_1DPT
dpm_fit['MTTF'] = MTTF_1DPM
dpb_fit['MTTF'] = MTTF_1DPB

mrm_heater_dpt_lifetime_estimation = MTTF_1DPB * A_lifetime[0]
mrm_heater_dpm_lifetime_estimation = MTTF_1DPB * A_lifetime[1]
mrm_heater_dpb_lifetime_estimation = MTTF_1DPB * A_lifetime[2]

crr_heater_dpt_lifetime_estimation = MTTF_1DPB * A_lifetime[0] * 10**2 / 4**2
crr_heater_dpm_lifetime_estimation = MTTF_1DPB * A_lifetime[1] * 10**2 / 4**2
crr_heater_dpb_lifetime_estimation = MTTF_1DPB * A_lifetime[2] * 10**2 / 4**2

dpt_fit['mrm_heater_lifetime_estimation'] = mrm_heater_dpt_lifetime_estimation
dpm_fit['mrm_heater_lifetime_estimation'] = mrm_heater_dpm_lifetime_estimation
dpb_fit['mrm_heater_lifetime_estimation'] = mrm_heater_dpb_lifetime_estimation

dpt_fit['crr_heater_lifetime_estimation'] = crr_heater_dpt_lifetime_estimation
dpm_fit['crr_heater_lifetime_estimation'] = crr_heater_dpm_lifetime_estimation
dpb_fit['crr_heater_lifetime_estimation'] = crr_heater_dpb_lifetime_estimation

print("Failure rate for 1DPT, 1DPM, 1DPB")
print(P_failure_rate)
print("A_lifetime for 1DPT, 1DPM, 1DPB")
print(A_lifetime)

reliability_data = pd.concat([dpt_fit, dpm_fit, dpb_fit], axis=0)
reliability_data.head()

[0.63097839 0.39810718 0.25118864]
Failure rate for 1DPT, 1DPM, 1DPB
[1.e-03 1.e-06 1.e-09]
A_lifetime for 1DPT, 1DPM, 1DPB
[0.63097839 0.39810718 0.25118864]


,heater_temp,heater_lifetime,probability,MTTF,mrm_heater_lifetime_estimation,crr_heater_lifetime_estimation
0,200.0,1430.148066,1DPT,2266.556350,1684.212277,10526.326730
1,200.1,1419.250781,1DPT,2249.285892,1671.410059,10446.312866
2,200.2,1408.436529,1DPT,2232.147030,1658.705154,10366.907212
3,200.3,1397.704679,1DPT,2215.138761,1646.096823,10288.105145
4,200.4,1387.054602,1DPT,2198.260089,1633.584332,10209.902077


In [3]:
px.line(reliability_data, x = "heater_temp", y="mrm_heater_lifetime_estimation", color="probability", title="MRM Heater Lifetime prediction vs MRM Heater Temperature", log_y = True, labels={"heater_temp":"MRM Heater Temperature (degC)", "lifetime_estimation":"MRM Heater Lifetime (years)"}, width=800, height=400)

In [4]:
px.line(reliability_data, x = "heater_temp", y="heater_lifetime", color="probability", title="MRM Heater Lifetime vs MRM Heater Temperature", log_y = True, labels={"heater_temp":"MRM Heater Temperature (degC)", "heater_lifetime":"MRM Heater Lifetime (years)"}, width=800, height=400)

In [5]:
px.line(reliability_data, x = "heater_temp", y="crr_heater_lifetime_estimation", color="probability", title="CRR Heater Lifetime prediction vs CRR Heater Temperature", log_y = True, labels={"heater_temp":"CRR Heater Temperature (degC)", "crr_heater_lifetime_estimation":"CRR Heater Lifetime (years)"}, width=800, height=400)

In [6]:
px.line(reliability_data, x = "heater_temp", y="MTTF", color="probability", title="MTTF vs MRM Heater Temperature", log_y = True, labels={"heater_temp":"MRM Heater Temperature (degC)", "MTTF":"MRM Heater MTTF (years)"}, width=800, height=400)

In [7]:
fatigue_factor = pd.read_csv(os.getcwd()+'/figure_5_fatigue_factor.csv')
fatigue_factor["Percentage"] = fatigue_factor["Percentage"]

In [8]:
px.scatter(fatigue_factor, x = "Time to Fail (a.u.)", y="Percentage", title="Time to Fail (a.u.) vs Percentage", log_x = True, log_y = True, labels={"Time to Fail (a.u.)":"Time to Fail (a.u.)", "Percentage":"% fail"}, width=800, height=400)

In [9]:
1-np.exp(-1*0.5**3.68)

0.075054682382523

In [10]:
np.log10(-1*np.log(1-0.075))/np.log10(0.5)

3.6810935736693864